# Chapter 2. Patterns for Cleaner Python
## 2.1 Covering Your A** With Assertions

In [11]:
import traceback as tb
from pprint import pprint

In [12]:
def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price']
    return price

In [13]:
shoes = {'name': 'Fancy Shoes', 'price': 14900}
apply_discount(shoes, 0.25)

11175

In [14]:
try:
    apply_discount(shoes, 2.0)
except AssertionError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

AssertionError()


  File "<ipython-input-14-d417721af32c>", line 2, in <module>
    apply_discount(shoes, 2.0)
  File "<ipython-input-12-bef2dce19d00>", line 3, in apply_discount
    assert 0 <= price <= product['price']


In [15]:
assert(1 == 2, 'This should fail')

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-15-2c057bd7fe24>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(1 == 2, 'This should fail')


## 2.2 Complacent Comma Placement

In [16]:
names = ['Alice', 'Bob', 'Dilbert']
names

['Alice', 'Bob', 'Dilbert']

In [17]:
names = [
    'Alice',
    'Bob',
    'Dilbert',
]
names

['Alice', 'Bob', 'Dilbert']

## 2.3 Context Managers and the with Statement
### Supporting with in Your Own Objects

In [18]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

In [19]:
with ManagedFile('hello.txt') as f:
    f.write('hello, world!\n')
    f.write('bye now')

In [20]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

In [21]:
with managed_file('hello.txt') as f:
    f.write('hello, world!\n')
    f.write('bye now')

### Writing Pretty APIs With Context Managers

In [22]:
class Indenter:
    def __init__(self):
        self.level = 0
    def __enter__(self):
        self.level += 1
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
    def print(self, text):
        print(' ' * self.level + text)

In [23]:
with Indenter() as indent:
    indent.print('hi!')  
    with indent:
        indent.print('hello')
        with indent:
            indent.print('bonjour')
    indent.print('hey')

 hi!
  hello
   bonjour
 hey


### page 35
If you’re looking for another exercise to deepen your understanding, try implementing a context manager that measures the execution time of a code block using the **time.time** function. Be sure to try out writing both a decorator-based and a class-based variant to drive home the difference between the two.

In [24]:
class TimeMeasurer():    
    from time import time
    global time
    
    def __init__(self):
        self.state = 0
    
    def __enter__(self):
        self.state += time()
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.state = time() - self.state
        print(f"execution time: {round(self.state, 2)} sec")

In [25]:
from time import sleep

In [26]:
with TimeMeasurer() as tm:
    sleep(2.5)

execution time: 2.5 sec


In [27]:
from contextlib import contextmanager

@contextmanager
def time_measurer():
    from time import time
    
    state = 0
    try:
        state += time()
        yield
    finally:
        state = time() - state
        print(f"execution time: {round(state, 2)} sec")

In [28]:
with time_measurer() as tm:
    sleep(2.5)

execution time: 2.5 sec


## 2.4 Underscores, Dunders, and More
### 1. Single Leading Underscore: “_var”

In [29]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23

In [30]:
test = Test()
test.foo

11

In [31]:
test._bar

23

In [32]:
from my_module import *

In [33]:
external_func()

23

In [34]:
try:
    _internal_func()
except NameError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

NameError("name '_internal_func' is not defined")


  File "<ipython-input-34-39de43e4e7fa>", line 2, in <module>
    _internal_func()


In [36]:
import my_module
my_module._internal_func()

42

### 2. Single Trailing Underscore: “var_”

In [40]:
# def make_object(name, class):
#     pass
# SyntaxError: "invalid syntax"

In [41]:
def make_object(name, class_):
    pass

### 3. Double Leading Underscore: “__var”

In [42]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        self.__baz = 23

In [43]:
test = Test()
dir(test)

['_Test__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

In [44]:
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'

In [45]:
t2 = ExtendedTest()
t2.foo

'overridden'

In [46]:
t2._bar

'overridden'

In [50]:
try:
    t2.__baz
except AttributeError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

AttributeError("'ExtendedTest' object has no attribute '__baz'")


  File "<ipython-input-50-de097a2ba791>", line 2, in <module>
    t2.__baz


In [51]:
dir(t2)

['_ExtendedTest__baz',
 '_Test__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

In [52]:
t2._ExtendedTest__baz

'overridden'

In [53]:
t2._Test__baz

23

In [55]:
class ManglingTest:
    def __init__(self):
        self.__mangled = 'hello'
    
    def get_mangled(self):
        return self.__mangled

In [56]:
ManglingTest().get_mangled()

'hello'

In [58]:
try:
    ManglingTest().__mangled
except AttributeError as e:
    print(repr(e))
    tb.print_tb(e.__traceback__)

AttributeError("'ManglingTest' object has no attribute '__mangled'")


  File "<ipython-input-58-e4625d45606b>", line 2, in <module>
    ManglingTest().__mangled


In [59]:
_MangledGlobal__mangled = 23

class MangledGlobal:
    def test(self):
        return __mangled

In [60]:
MangledGlobal().test()

23

### 4. Double Leading and Trailing Underscore: "\_\_var\_\_"

In [72]:
class PrefixPostfixTest:
    def __init__(self):
        self.__bam__ = 42

In [73]:
PrefixPostfixTest().__bam__

42

### 5. Single Underscore: "\_"

In [75]:
for _ in range(3):
    print("Hello, World!")

Hello, World!
Hello, World!
Hello, World!


In [76]:
car = ('red', 'auto', 12, 3812.4)
color,_,_,mileage = car

In [77]:
color, mileage

('red', 3812.4)

## 2.5 A Shocking Truth About String Formatting

In [78]:
errno = 50159747054
name = 'Bob'

In [80]:
f"Hey {name}, there is a {errno:#x} error!"

'Hey Bob, there is a 0xbadc0ffee error!'

In [81]:
a = 5
b = 10
f'Five plus ten is {a + b} and not {2 * (a + b)}.'

'Five plus ten is 15 and not 30.'

In [82]:
def greet(name, question):
    return f"Hello, {name}! How's it {question}?"

In [83]:
import dis
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 FORMAT_VALUE             0
              6 LOAD_CONST               2 ("! How's it ")
              8 LOAD_FAST                1 (question)
             10 FORMAT_VALUE             0
             12 LOAD_CONST               3 ('?')
             14 BUILD_STRING             5
             16 RETURN_VALUE


### #4 – Template Strings

In [84]:
from string import Template
t = Template('Hey, $name!')
t.substitute(name=name)

'Hey, Bob!'

In [85]:
templ_string = 'Hey $name, there is a $error error!'
Template(templ_string).substitute(name=name, error=hex(errno))

'Hey Bob, there is a 0xbadc0ffee error!'

In [86]:
SECRET = 'this-is-a-secret'

In [87]:
class Error:
    def __init__(self):
        pass

In [88]:
err = Error()
user_input = '{error.__init__.__globals__[SECRET]}'

In [89]:
user_input.format(error=err)

'this-is-a-secret'

In [97]:
globals()['SECRET']

'this-is-a-secret'

In [99]:
user_input = '${error.__init__.__globals__[SECRET]}'

try:
    Template(user_input).substitute(error=err)
except ValueError as e:
    print(repr(e))
    print(tb.print_tb(e.__traceback__))

ValueError('Invalid placeholder in string: line 1, col 1')
None


  File "<ipython-input-99-2b08d6a27392>", line 4, in <module>
    Template(user_input).substitute(error=err)
  File "c:\program files\python38\lib\string.py", line 126, in substitute
    return self.pattern.sub(convert, self.template)
  File "c:\program files\python38\lib\string.py", line 123, in convert
    self._invalid(mo)
  File "c:\program files\python38\lib\string.py", line 106, in _invalid
    raise ValueError('Invalid placeholder in string: line %d, col %d' %


### 2.6 “The Zen of Python” Easter Egg

In [101]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
